In [ ]:
import transformers
from transformers import pipeline

summarizer = pipeline(task="summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
summarizer(
    "In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention. For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. In the former task our best model outperforms even all previously reported ensembles."
)

Your max_length is set to 142, but your input_length is only 117. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)


[{'summary_text': ' The Transformer is the first sequence transduction model based entirely on attention . It replaces the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention . For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers .'}]

In [ ]:
!kaggle datasets download -d nileshmalode1/samsum-dataset-text-summarization

Dataset URL: https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization
License(s): unknown
 63% 5.00M/7.99M [00:00<00:00, 33.6MB/s]
100% 7.99M/7.99M [00:00<00:00, 47.6MB/s]


In [ ]:
!unzip -o samsum-dataset-text-summarization.zip

Archive:  samsum-dataset-text-summarization.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
  inflating: samsum_dataset/dataset_dict.json  
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data=pd.read_csv("/content/samsum-train.csv")
test_data=pd.read_csv("/content/samsum-test.csv")
val_data=pd.read_csv("/content/samsum-validation.csv")

In [ ]:
train_data.shape,test_data.shape,val_data.shape

((14732, 3), (819, 3), (818, 3))

In [ ]:
train_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [ ]:
# as it is very large so just use some data ka part
train_data=train_data.sample(n=6000,random_state=42).reset_index(drop=True)
test_data=test_data.sample(n=500,random_state=42).reset_index(drop=True)
val_data=val_data.sample(n=500,random_state=42).reset_index(drop=True)

In [ ]:
train_data.shape


(6000, 3)

In [ ]:
train_data.head()

,id,dialogue,summary
0,13811908,Violet: hi! i came across this Austin's articl...,Violet sent Claire Austin's article.
1,13716431,Pat: So does anyone know when the stream is go...,Pat and Lou are waiting for The stream but Kev...
2,13810214,Jane: <gif_file>\r\nJane: Whaddya think? \r\nS...,Jane is updating her Tinder profile tonight an...
3,13729823,"Adam: Do u have a map of Paris?\r\nTom: Yes, W...",Tom has a map of Paris.
4,13681400,"Frank: Hi, how's the family?\r\nMike: great! S...","Mike is happy, because Sam's moved out. Mike a..."


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()

    # 2. Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 3. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # 4. Remove emails
    text = re.sub(r'\S+@\S+', '', text)

    # 5. Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # 6. Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # 7. Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 8. Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # here stopwords dont remove as claiisif ynhi krna hai sumarise jrna hai
  #  as here we want all words not special words


    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
train_data.dialogue=train_data.dialogue.apply(preprocess_text)
train_data.summary=train_data.summary.apply(preprocess_text)

In [ ]:
val_data.dialogue=val_data.dialogue.apply(preprocess_text)
val_data.summary=val_data.summary.apply(preprocess_text)

In [ ]:
test_data.dialogue=test_data.dialogue.apply(preprocess_text)
test_data.summary=test_data.summary.apply(preprocess_text)

In [ ]:
train_data.iloc[0:10]

,id,dialogue,summary
0,13811908,violet hi i came across this austins article a...,violet sent claire austins article
1,13716431,pat so does anyone know when the stream is goi...,pat and lou are waiting for the stream but kev...
2,13810214,jane jane whaddya think shona this ur tinder p...,jane is updating her tinder profile tonight an...
3,13729823,adam do u have a map of paris tom yes why a tr...,tom has a map of paris
4,13681400,frank hi hows the family mike great sams moved...,mike is happy because sams moved out mike and ...
5,13716070,paul lucky you john pete our classess have bee...,john pete and pauls classes have been cancelled
6,13727976,jasper i miss you so much already karen i know...,karen will be back on sunday karen and jasper ...
7,13681231,ken how long do you need jude i think about mi...,ken will wait inside as jude needs more minutes
8,13862652,victoria hey i am in the toiletand skylar i kn...,victoria is in a restaurant toilet and texts s...
9,13728508,sandra do u need any help with the party tomor...,ronda does not need any help with the party to...


In [ ]:
train_data.dialogue[0],train_data.summary[0]

('violet hi i came across this austins article and i thought that you might find it interesting violet claire hi thanks but ive already read it claire but thanks for thinking about me',
 'violet sent claire austins article')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer,Trainer, TrainingArguments

# Specify the model size ('t5-small', 't5-base', 't5-large', etc.)
model_name = "t5-small"

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Truncation in the context of tokenization refers to the process of limiting the length of the tokenized sequence to a specified maximum length by removing excess tokens. This ensures that input sequences do not exceed the model's maximum allowable input size, which can vary by model architecture (e.g., 512 tokens for many transformer models like BERT, T5, and GPT).

Key Points About Truncation:
Why It's Needed:

Transformer models like T5 or BERT have a maximum sequence length they can handle (e.g., 512 tokens). Any sequence longer than this needs to be truncated to avoid errors during model inference or training.
How It Works:

If a tokenized input sequence exceeds the max_length specified during tokenization, the tokenizer will discard tokens from the end of the sequence until it fits within the max_length.
Specifying Truncation in Hugging Face:

The truncation=True argument in the tokenizer ensures that sequences are truncated automatically during tokenization.

In [ ]:
t=train_data.copy()
d=train_data.copy()

In [ ]:
t["tokenized_length"] = t["dialogue"].apply(
    lambda x: len(tokenizer.encode(x, truncation=False))
)
max_length = t["tokenized_length"].max()
print(f"Maximum tokenized length: {max_length}")
# so it give 526 as without truncate

Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors


Maximum tokenized length: 987


In [ ]:
d["tokenized_length"] = d["dialogue"].apply(
    lambda x: len(tokenizer.encode(x, truncation=True))
)
max_length = d["tokenized_length"].max()
print(f"Maximum tokenized length: {max_length}")
# it give 512 with truncate

Maximum tokenized length: 512


In [ ]:

# Preprocessing function for tokenization
def preprocess_function(examples):
    # Tokenize the dialogue and summary
    inputs = tokenizer(examples["dialogue"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply the preprocessing
train_dataset = train_data.apply(preprocess_function, axis=1)
val_dataset = val_data.apply(preprocess_function, axis=1)
test_dataset = test_data.apply(preprocess_function, axis=1)

An attention mask is a mechanism used in transformer-based models to differentiate between actual input tokens and padding tokens. It ensures that the model attends only to meaningful tokens and ignores the padding during computation.

Key Points about Attention Mask:
Purpose:

Mask padding tokens so they do not influence the model's attention mechanism.
Allows the model to focus only on valid tokens in variable-length sequences.
How It Works:

The attention mask is a binary array of the same shape as the input.
Each position corresponds to a token in the input:
1: The token is valid (should be attended to).
0: The token is padding (should be ignored).
When It's Used:

In training or inference when sequences are padded to a fixed length.
Padding is common in batching, where all sequences in a batch must have the same length.

In [ ]:
'''Input sentences'''
'''sentences = ["This is a short sentence.",
             "This is another sentence, but longer."]
Input IDs:
tensor([[  71,   19,    3,  230,   10,    1,    0,    0,    0,    0],
        [  71,   19, 1326,   10,   19, 1314,    1,    0,    0,    0]])
#1 in tensors after 10 and 1314 represent end of sentence then uske baad paddin wae zerores
Attention Mask:
# 1 for real tokens and 0 for padding part
tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])'''




'sentences = ["This is a short sentence.", \n             "This is another sentence, but longer."]\nInput IDs: \ntensor([[  71,   19,    3,  230,   10,    1,    0,    0,    0,    0],\n        [  71,   19, 1326,   10,   19, 1314,    1,    0,    0,    0]])\n#1 in tensors after 10 and 1314 represent end of sentence then uske baad paddin wae zerores\nAttention Mask:\n# 1 for real tokens and 0 for padding part\ntensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],\n        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])'

fter tokenizing both the "dialogue" and "summary", the inputs dictionary contains the tokenized input (dialogue) tokens in the form of input_ids (i.e., the token IDs).

The targets dictionary contains the tokenized target (summary) tokens in the form of input_ids. These are the tokenized target values that we want the model to predict.

inputs["labels"] = targets["input_ids"]:

Here, we are adding the input_ids from the targets dictionary as the labels for the model.
In transformer-based models, the labels field holds the target sequence (the correct output) that the model tries to predict. For a task like summarization, this is the sequence that the model learns to generate when given the "dialogue" as input.
By assigning targets["input_ids"] to inputs["labels"], we essentially make the tokenized summary (the target) the correct output that the model tries to match during training.

In [ ]:
train_dataset[0] # so labels are our output that is summarise text

{'input_ids': [25208, 7102, 3, 23, 764, 640, 48, 403, 17, 77, 7, 1108, 11, 3, 23, 816, 24, 25, 429, 253, 34, 1477, 25208, 3, 7997, 15, 7102, 2049, 68, 3, 757, 641, 608, 34, 3, 7997, 15, 68, 2049, 21, 1631, 81, 140, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
val_dataset[0]

{'input_ids': [3, 15, 26, 26, 2275, 210, 410, 25, 1616, 24, 79, 60, 3, 23560, 178, 12, 3, 9, 315, 3066, 4659, 14228, 9, 9, 9, 9, 144, 3, 32, 4659, 150, 213, 26, 25, 1616, 24, 3, 15, 26, 26, 168, 165, 882, 2314, 3, 15, 26, 26, 11, 13515, 131, 1219, 178, 4659, 11, 103, 25, 214, 125, 34, 1112, 21, 178, 3, 15, 26, 26, 79, 751, 17, 483, 8, 5812, 7, 3, 15, 26, 26, 68, 3, 23, 214, 8, 16879, 56, 129, 7873, 972, 4659, 11, 3, 23, 3382, 24, 19, 3, 9, 888, 24, 19, 5741, 12, 143, 762, 1842, 3, 15, 26, 26, 17945, 3382, 78, 3, 15, 26, 26, 79, 43, 3, 9, 6613, 194, 13, 1705, 12, 143, 378, 1842, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
max(len(x) for x in train_data.dialogue)

3901

In [ ]:
max(len(tokenizer.encode(x)) for x in train_data.dialogue)

987

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",          # output directory for checkpoints
    num_train_epochs=6,              # number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate alpha in backpropogstion scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=50,                # how often to log training info
    save_steps=500,                  # how often to save a model checkpoint
    eval_steps=50,                   # how often to run evaluation
    evaluation_strategy="epoch",     # Ensure evaluation happens every `epoch`
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.450400,0.390128
2,0.410500,0.372112
3,0.388900,0.365431
4,0.387600,0.361004
5,0.389700,0.359627
6,0.356000,0.359142


TrainOutput(global_step=4500, training_loss=0.8048931079440647, metrics={'train_runtime': 1805.3774, 'train_samples_per_second': 19.94, 'train_steps_per_second': 2.493, 'total_flos': 4872304852992000.0, 'train_loss': 0.8048931079440647, 'epoch': 6.0})

In [ ]:
# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./saved_summary_model")
tokenizer = T5Tokenizer.from_pretrained("./saved_summary_model")

In [ ]:
device = model.device  # Get the device the model is on GPU/CPU

def summarize_dialogue(dialogue):
    dialogue = preprocess_text(dialogue)  # Assuming preprocess_text is defined
    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate summary
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,  #outptu summary ki length
        num_beams=4, #give top 4 summary of a text ki top closest summmary
        early_stopping=True# jitni ability hai train hone ki uske aage nhi badhna hai
    )

    # Decode the generated summary
    # <ELO> this is my book <ELO> # so it special keyqord <ELO> jo rhega output mein jo inko ignore krke beech wala sentend do
    # skip_special_token=true
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
sample_dialogue = """
Violet: Hey Claire! I was reading an article about Austin and thought you might find it interesting!
Violet: It's about the current trends in urban development and how cities are planning for the future.
Violet: Here, let me share the link: <file_other>
Claire: Oh wow, that sounds like an insightful read. But I've actually already read that one last week.
Claire: It was really interesting though, especially the part about sustainable architecture in cities.
Claire: You know, I've been following these urban planning discussions for a while now.
Violet: Oh, I didn’t know that! Well, I’ll look for something else then, maybe something about eco-friendly cities or tech innovations.
Claire: That would be awesome! Let me know if you find something cool.
Violet: Sure, I’ll keep you posted. Thanks for the feedback!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: claire was reading an article about austin and thought she might find it interesting


In [ ]:
sample_dialogue = """
John: Hey Sarah, have you seen the latest tech gadget reviews? I found this new smartwatch that's supposed to have amazing health tracking features.
John: It tracks heart rate, blood oxygen levels, sleep patterns, and even stress levels! It sounds like something right up your alley.
Sarah: That sounds really interesting! But I’ve been trying to cut down on tech distractions. I’ve heard these devices can be really overwhelming sometimes.
Sarah: I do think it’s cool that they can track so many health metrics though. I’m curious how accurate they really are.
John: Yeah, me too! There are also some new smartphones coming out with even better cameras and longer battery life. The new flagship model from XYZ brand has some insane specs.
Sarah: Ooh, I haven’t kept up with phones recently, but I’ve heard the camera quality is getting ridiculously good. It’s almost like a professional camera in your pocket now!
Sarah: Still, I feel like I’m fine with my current phone for now. I don’t really feel the need to upgrade unless something really groundbreaking comes out.
John: Totally understand that. It’s the same with me. But I think the battery life improvements are enough to make me consider it. I hate running out of battery when I’m out and about.
Sarah: That’s fair! I’m always worried about battery life too. Honestly, I think phones should last at least two full days on a single charge by now.
John: I agree! It’s so annoying when your phone dies in the middle of the day. I wonder if we’ll ever get to a point where we don’t have to charge our phones every day.
Sarah: That would be amazing! I think as tech improves, battery tech might also catch up. Let’s hope the next generation of phones can last longer!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: john found a new smartwatch that tracks heart rate blood oxygen levels sleep patterns and stress levels sarah hasnt kept up with phones recently but hasnt kept up with phones yet john thinks the battery life improvements are enough to make her consider it sarah hates running out of battery


In [ ]:
# Test with a dialogue on a current news topic
sample_dialogue = """
Reporter: In today's news, the latest climate change report reveals alarming global temperature rises. According to the Intergovernmental Panel on Climate Change (IPCC), the Earth’s temperature is on track to rise by 1.5°C within the next two decades.
Reporter: This is expected to lead to more frequent and severe heatwaves, flooding, and extreme weather events. Coastal cities are at particular risk due to rising sea levels.
Expert: The report emphasizes that immediate action is needed to prevent catastrophic consequences. We need to significantly reduce carbon emissions and transition to renewable energy sources.
Expert: If global temperatures increase by more than 1.5°C, we could face irreversible damage to ecosystems, agriculture, and water supply. It will have a devastating impact on biodiversity as well.
Reporter: The IPCC also stresses the importance of individual action. Governments must set stronger policies, but individuals can help by reducing waste, conserving water, and supporting green initiatives.
Expert: It's not just about the big changes; small actions like using public transportation, reducing meat consumption, and recycling can collectively make a significant difference.
Reporter: With the next UN Climate Summit coming up next month, world leaders will need to prioritize climate action. The stakes have never been higher for our planet’s future.
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: ipcc report reveals alarming global temperature rises according to the intergovernmental panel on climate change ipcc the earths temperature is on track to rise by c within the next two decades this is expected to lead to more frequent and severe heatwaves flooding and extreme weather events coastal cities are at particular risk due to rising sea levels


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Save the trained model
model.save_pretrained("saved_summary_model")

# Save the tokenizer
tokenizer.save_pretrained("saved_summary_model")


('saved_summary_model/tokenizer_config.json',
 'saved_summary_model/special_tokens_map.json',
 'saved_summary_model/spiece.model',
 'saved_summary_model/added_tokens.json')

In [ ]:
import shutil

# Compress the directory into a zip file
shutil.make_archive("saved_summary_model", 'zip', "saved_summary_model")


'/content/saved_summary_model.zip'

In [ ]:
from google.colab import files

# Download the zip file
files.download("saved_summary_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Download the file
files.download('/content/saved_summary_model/model.safetensors')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>